In [4]:
#__________________TASK 1 - Data prepration-___________________
import pandas as pd
import sqlite3

# -----------------------------
# Step 1: Load excel data
# -----------------------------
df= pd.read_excel('/Users/itadministrator/Desktop/PWC Project/PWC Project 2 /fmcg_mult_prod_transactions_lifecycle_sample - Copy.xlsx')

#---------date time conversion---------#
df['transaction_month'] = pd.to_datetime(df['transaction_month'], format='%Y-%m')

# ------------------------Remove duplicate transactions--------------------
df.drop_duplicates(subset='transaction_id', inplace=True)

#-------------re-calculating sales amt for accuracy------
df['sales_amount']= df['unit_price']*df['volume_units']

#--------missing values-------
df.dropna(inplace=True)
# --------------------------------
# Step 3: Store in SQLite
# --------------------------------
conn = sqlite3.connect("sales_data.db")

df.to_sql(
    name="sales_data",
    con=conn,
    if_exists="replace",
    index=False
)

conn.commit()

#------checking the dataset again------#
df.info()
df.describe()
df.isnull().sum()

#Monthly Product-Level Aggregation
monthly_product_sales = (
    df.groupby(['transaction_month', 'product_id', 'product_name'])
      .agg(
          total_sales=('sales_amount', 'sum'),
          total_units=('volume_units', 'sum')
      )
      .reset_index()
)

#Sort for Time-Series Analysis
monthly_product_sales = monthly_product_sales.sort_values('transaction_month')


<class 'pandas.core.frame.DataFrame'>
Index: 27666 entries, 0 to 82946
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   transaction_id     27666 non-null  object        
 1   customer_number    27666 non-null  int64         
 2   transaction_month  27666 non-null  datetime64[ns]
 3   product_id         27666 non-null  object        
 4   product_name       27666 non-null  object        
 5   unit_price         27666 non-null  float64       
 6   volume_units       27666 non-null  int64         
 7   sales_amount       27666 non-null  float64       
 8   promo_offer        27666 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 2.1+ MB


In [5]:
# Task Clean and transform data for analysis, including aggregation by time periods and product categories.

# ============================================
# TASK 1 (Part 2): Clean & Transform Data
# ============================================

import pandas as pd

# --------------------------------------------
# 1.Time-Based Feature Engineering
# --------------------------------------------

# Extract Year, Month, Quarter
df['year'] = df['transaction_month'].dt.year
df['month'] = df['transaction_month'].dt.month
df['quarter'] = df['transaction_month'].dt.quarter

# Create Year-Month Period (for monthly trend analysis)
df['year_month'] = df['transaction_month'].dt.to_period('M')


# --------------------------------------------
# 2.Monthly Aggregation
# --------------------------------------------

monthly_sales = (
    df.groupby('year_month')
      .agg(
          total_sales=('sales_amount', 'sum'),
          total_units=('volume_units', 'sum'),
          total_transactions=('transaction_id', 'nunique')
      )
      .reset_index()
)

# Convert Period to string for visualization / chatbot output
monthly_sales['year_month'] = monthly_sales['year_month'].astype(str)


# --------------------------------------------
# 3. Quarterly Aggregation
# --------------------------------------------

quarterly_sales = (
    df.groupby(['year', 'quarter'])
      .agg(
          total_sales=('sales_amount', 'sum'),
          total_units=('volume_units', 'sum'),
          total_transactions=('transaction_id', 'nunique')
      )
      .reset_index()
)


# --------------------------------------------
# 4.Product-Level Aggregation
# --------------------------------------------

product_sales = (
    df.groupby(['product_id', 'product_name'])
      .agg(
          total_sales=('sales_amount', 'sum'),
          total_units=('volume_units', 'sum'),
          total_transactions=('transaction_id', 'nunique')
      )
      .reset_index()
      .sort_values('total_sales', ascending=False)
)


# --------------------------------------------
# 5.Dynamic Product Category Creation
# --------------------------------------------

# If product_category does NOT exist, derive from product_name
df['product_category'] = df['product_name'].apply(lambda x: x.split()[0])


# --------------------------------------------
# 6.Category-Level Aggregation
# --------------------------------------------

category_sales = (
    df.groupby('product_category')
      .agg(
          total_sales=('sales_amount', 'sum'),
          total_units=('volume_units', 'sum'),
          total_transactions=('transaction_id', 'nunique')
      )
      .reset_index()
      .sort_values('total_sales', ascending=False)
)


# --------------------------------------------
# 7.Sort Monthly Data for Time-Series Analysis
# --------------------------------------------

monthly_sales = monthly_sales.sort_values('year_month')


# --------------------------------------------
# 8.Display Summary (Optional Validation)
# --------------------------------------------

print("Monthly Sales Preview:")
print(monthly_sales.head())

print("\nQuarterly Sales Preview:")
print(quarterly_sales.head())

print("\nTop Products Preview:")
print(product_sales.head())

print("\nCategory Sales Preview:")
print(category_sales.head())

Monthly Sales Preview:
  year_month  total_sales  total_units  total_transactions
0    2022-01    534857.89         1900                 334
1    2022-02   1045210.22         3826                 682
2    2022-03   1406513.60         5073                 934
3    2022-04   1697794.22         6368                1164
4    2022-05   1833233.72         6821                1260

Quarterly Sales Preview:
   year  quarter  total_sales  total_units  total_transactions
0  2022        1   2986581.71        10799                1950
1  2022        2   5445640.18        20286                3720
2  2022        3   6089698.49        22123                4036
3  2022        4   6005561.97        21768                3979
4  2023        1   6247501.69        22939                4154

Top Products Preview:
   product_id            product_name  total_sales  total_units  \
11       P012    Godrej Shampoo 200ml   2150511.59         3847   
34       P035  Milky Mist Paneer 200g   2100212.10         377

In [3]:
# ============================================
#Task 2:
#Section 1 Chatbot Core Analytics Engine
# ============================================

import sqlite3
import pandas as pd
from datetime import datetime
import time

# --------------------------------------------
# Database Path
# --------------------------------------------

DB_PATH = "sales_data.db"


# --------------------------------------------
# Function: Execute SQL Query
# --------------------------------------------

def execute_sql(query):
    conn = sqlite3.connect(DB_PATH)
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df


# --------------------------------------------
# Function: Interpret User Query
# --------------------------------------------

def interpret_query(user_query):
    query = user_query.lower()

    # 1.Total Revenue
    if "total revenue" in query:
        return """
        SELECT ROUND(SUM(sales_amount), 2) AS total_revenue
        FROM sales_data;
        """

    # 2.Monthly Sales Trend
    elif "monthly" in query and "sales" in query:
        return """
        SELECT strftime('%Y-%m', transaction_month) AS month,
               ROUND(SUM(sales_amount), 2) AS total_sales
        FROM sales_data
        GROUP BY month
        ORDER BY month;
        """

    # 3.Top 5 Products
    elif "top" in query and "product" in query:
        return """
        SELECT product_name,
               ROUND(SUM(sales_amount), 2) AS total_sales
        FROM sales_data
        GROUP BY product_name
        ORDER BY total_sales DESC
        LIMIT 5;
        """

    # 4.Promotion Analysis
    elif "promotion" in query or "promo" in query:
        return """
        SELECT promo_offer,
               ROUND(SUM(sales_amount), 2) AS total_sales,
               SUM(volume_units) AS total_units
        FROM sales_data
        GROUP BY promo_offer;
        """

    else:
        return None


# ============================================
# Task 2 – Section 2
# Implement functions to determine question categories – whether it is a descriptive question or a visualization question
# ============================================

def classify_question(user_query):
    """
    Determines whether a user query is:
    - 'descriptive'
    - 'visualization'
    """

    query = user_query.lower().strip()

    visualization_keywords = [
        "show", "plot", "chart", "graph",
        "visualize", "trend", "compare",
        "display"
    ]

    if any(keyword in query for keyword in visualization_keywords):
        return "visualization"

    return "descriptive"

# ============================================
# Task 2 – Section 3
# Enable dynamic generation of textual insights based on the data obtained.
# ============================================

def generate_textual_insight(user_query, result_df):
    """
    Generates human-readable insights dynamically
    based on the query result DataFrame.
    """

    if result_df is None or result_df.empty:
        return "No data available to generate insights."

    query = user_query.lower()

    #Total Revenue Insight
    if "total revenue" in query:
        revenue = result_df.iloc[0, 0]
        return (
            f"The total revenue generated is ₹{revenue:,.2f}. "
            f"This reflects the overall financial performance of the retail store."
        )

    #Monthly Sales Insight
    elif "monthly" in query and "sales" in query:
        peak_row = result_df.loc[result_df.iloc[:, 1].idxmax()]
        return (
            f"Sales peaked in {peak_row.iloc[0]} with revenue of "
            f"₹{peak_row.iloc[1]:,.2f}. "
            f"This indicates the strongest performing month."
        )

    #Top Product Insight
    elif "top" in query and "product" in query:
        top_product = result_df.iloc[0]
        return (
            f"The top-performing product is '{top_product.iloc[0]}' "
            f"with total revenue of ₹{top_product.iloc[1]:,.2f}. "
            f"This product significantly contributes to overall sales."
        )

    #Promotion Insight
    elif "promotion" in query or "promo" in query:
        best_promo = result_df.loc[result_df.iloc[:, 1].idxmax()]
        return (
            f"The '{best_promo.iloc[0]}' promotion generated the highest revenue "
            f"of ₹{best_promo.iloc[1]:,.2f}. "
            f"This suggests promotional strategies positively impact sales."
        )

    return "Analysis completed successfully."

In [ ]:
#---------------------------------------------
# Task 2 – Section 1
# Integrate GPT-4 or Gemini Pro APIs to understand user intent and generate contextual responses.
#---------------------------------------------


